In [1]:
import json
import pandas as pd
import numpy as np
import collections as col
from pathlib import Path
from requests_html import HTMLSession
from typing import List, Dict

## Data preparation

In [2]:
session = HTMLSession()

In [3]:
def crawl_data(_from: int, _to: int) -> Dict[str, List[str]]:
    results = {
        'title': [],
        'content': []
    }

    for page in range(_from, _to + 1):
        r = session.get(f"http://truyencotich.vn/truyen-co-tich/co-tich-viet-nam/page/{page}")

        for link in set([element.attrs["href"] for element in r.html.find("article a")]):
            page = session.get(link)
            
            title = page.html.find("#content .entry-title", first=True).text
            content = page.html.find("#content .entry-content", first=True).text

            results['title'].append(title)
            results['content'].append(content)
    
    return results

In [4]:
data_path = "./bed-time-stories.json"
file = Path(data_path)
data = None

In [5]:
if file.exists():
    data = json.load(open(data_path))
else:
    data = crawl_data(1, 30)

    with open(data_path, "w") as outfile:
        json.dump(data, outfile)

## Create pandas df

In [6]:
df = pd.DataFrame(data=data)

In [7]:
df.head()

,content,title
0,"Ngày xưa, có một người con gái con một nhà già...",Sự tích ông bình vôi
1,"Ngày xửa ngày xưa, ở một vùng nọ có đôi vợ chồ...",Anh học trò và ba con quỷ
2,"Ngày xưa, có nội tán Nguyễn Khoa Đăng là người...",Nguyễn Khoa Đăng
3,"Ngày xưa, ở dưới chân núi Hồng-lĩnh về phía Đô...",Cố Ghép
4,Ở làng kia có một người nung đất làm chén đĩa ...,Thày lang bất đắc dĩ


In [8]:
content = df.iloc[0].content

print(content)

Ngày xưa, có một người con gái con một nhà giàu có. Cô rất đẹp nhưng cũng rất kiêu. Cô đã từng làm cho các bạn gái xa lánh mình. Cô làm cho các chàng trai ghét cô vô hạn. Cũng vì thế đến tuổi lấy chồng, cô gái vẫn chưa có đám nào vừa ý. Chàng trai nào cũng bị cô sổ toẹt, vì cao chê ngỏng, thấp chê lùn, lớn chê béo trục béo tròn, gầy chê xương sống xương sườn bày ra.
Nhưng rồi cô cũng lấy được chồng. Chồng cô yêu vợ nhưng lại bực mình vì thói ghen của vợ. Cô ghen chồng làm cho xóm giềng luôn cau mặt vì những lời qua tiếng lại của họ. Cuối cùng hai người không vừa ý nhau và ly dị. Buồn bực vì duyên phận, cô bỏ đi tu.
Cô xuất gia ở một ngôi chùa cổ trên núi suốt hai mươi năm. Những con chim, con thú rừng hầu như quen thuộc bóng dáng của người sư nữ. Hai mươi năm qua, cô vẫn chưa đắc đạo. Cô thắc mắc, vì tự cho mình thông kinh kệ hơn người và chịu đủ mọi sự khổ hạnh của nhà chiền. Một ngày kia cô quyết định sang Tây trúc một phen để tìm cho ra lẽ. Đường sang Tây-trúc thiên nguy vạn hiểm nh

## Words processing

In [9]:
from collections import Counter

In [10]:
content_data = df.content.str.findall("\w+|[.,!?;]")
title_data = df.title.str.findall("\w+|[.,!?;]")

## Test with gensim

In [15]:
from gensim import corpora

In [16]:
dictionary = corpora.Dictionary(documents=content_data)

In [19]:
new_vec = dictionary.doc2bow(content.lower().split())
print(new_vec)

[(66, 1), (67, 1), (68, 1), (69, 2), (71, 3), (72, 8), (73, 1), (74, 1), (75, 4), (76, 4), (77, 2), (78, 2), (79, 4), (80, 1), (81, 3), (82, 1), (83, 5), (84, 1), (85, 2), (86, 4), (87, 3), (90, 1), (91, 5), (92, 2), (93, 2), (94, 1), (95, 1), (96, 2), (97, 2), (98, 1), (99, 3), (100, 2), (101, 2), (102, 3), (103, 1), (104, 1), (108, 30), (109, 1), (110, 2), (111, 2), (112, 3), (113, 4), (114, 4), (115, 5), (116, 11), (117, 2), (118, 1), (119, 1), (120, 1), (121, 3), (122, 3), (123, 1), (124, 1), (125, 1), (126, 1), (127, 5), (128, 1), (129, 1), (130, 5), (131, 3), (132, 1), (133, 1), (134, 1), (135, 2), (136, 16), (137, 2), (138, 2), (139, 1), (140, 1), (141, 2), (142, 3), (143, 3), (144, 1), (145, 6), (146, 3), (147, 18), (148, 15), (149, 3), (150, 2), (151, 17), (152, 1), (154, 1), (155, 1), (156, 1), (157, 1), (158, 1), (159, 1), (160, 4), (161, 3), (162, 13), (163, 2), (164, 1), (165, 1), (166, 3), (168, 1), (169, 1), (171, 1), (173, 1), (174, 1), (176, 1), (178, 1), (180, 1), (18